In [47]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
import pylab as pl
from sklearn.metrics import fbeta_score

In [2]:
ct_train=pd.read_csv('Data\carvan_train.csv')
ct_test=pd.read_csv('Data\carvan_test.csv')

In [3]:
ct_test['V86'] = np.nan
ct_train['data'] = 'train' 
ct_test['data'] = 'test'
ct_test = ct_test[ct_train.columns]
ct_all = pd.concat([ct_train,ct_test],axis=0)

In [4]:
(ct_train['V86'][ct_train['V86']==1].value_counts()/ct_train.shape[0])*100

1    5.977327
Name: V86, dtype: float64

In [5]:
##CReating dummies for categorical vars
def dummies(data,var,freq_cutoff=0):
    t= data[var].value_counts(normalize=True) 
    t=t[t.values>freq_cutoff] 
    t=t.sort_values() 
    t_min=t.idxmin()
    t=t.drop([t_min])
    categories=t.index

    for cat in categories :
        name=var+'_'+cat
        name=re.sub(" ","",name) 
        name=re.sub("-","_",name)
        name=re.sub("\\?","Q",name) 
        name=re.sub("<","LT_",name)
        name=re.sub("\\+","",name) 
        name=re.sub("\\/","_",name) 
        name=re.sub(">","GT_",name) 
        name=re.sub("=","EQ_",name)
        name=re.sub(",","",name)
        data[name]=(data[var]==cat)+0 
               
    data=data.drop(columns=[var])
    return data

In [6]:
for i in ct_all:
    if(i!='V86' and i!='data'):
        ct_all[i] = ct_all[i].astype(str)
        ct_all = dummies(ct_all,i)

In [7]:
ct_train=ct_all[ct_all['data']=='train'] 
del ct_train['data'] 
ct_test=ct_all[ct_all['data']=='test']
ct_test.drop(['V86','data'],axis=1,inplace=True) 

C:\Users\jayes\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [8]:
train1, train2 = train_test_split(ct_train, test_size = 0.2,random_state=2)

x_train1=train1.drop(["V86"],1)
y_train1=train1["V86"]
x_train2=train2.drop(["V86"],1)
y_train2=train2["V86"]

x_train1.reset_index(drop=True,inplace=True)
y_train1.reset_index(drop=True,inplace=True)

In [38]:
svclassifier = SVC(kernel='rbf', class_weight='balanced', C=1.0, random_state=0)
svclassifier.fit(x_train1, y_train1)

SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=0,
    shrinking=True, tol=0.001, verbose=False)

In [39]:
y_pred1 = svclassifier.predict(x_train1).round()

In [40]:
y_pred2 = svclassifier.predict(x_train2).round()

In [62]:
y_pred2[y_pred2==1].size

4657

In [42]:
print(confusion_matrix(y_train1,y_pred1))
print(classification_report(y_train1,y_pred1))

[[3138 1246]
 [  51  222]]
              precision    recall  f1-score   support

         0.0       0.98      0.72      0.83      4384
         1.0       0.15      0.81      0.26       273

    accuracy                           0.72      4657
   macro avg       0.57      0.76      0.54      4657
weighted avg       0.94      0.72      0.80      4657



In [43]:
print(confusion_matrix(y_train2,y_pred2))
print(classification_report(y_train2,y_pred2))

[[805 285]
 [ 22  53]]
              precision    recall  f1-score   support

         0.0       0.97      0.74      0.84      1090
         1.0       0.16      0.71      0.26        75

    accuracy                           0.74      1165
   macro avg       0.57      0.72      0.55      1165
weighted avg       0.92      0.74      0.80      1165



In [45]:
roc_auc_score(y_train1, y_pred1) #auc score of train 1

0.7644857423598299

In [46]:
roc_auc_score(y_train2, y_pred2) #auc score of train 1

0.7225993883792049

In [58]:
fbeta_score(y_train1, y_pred1, beta=2)

0.43359375

In [59]:
fbeta_score(y_train2, y_pred2, beta=2)

0.4153605015673981

In [72]:
y_test_pred = svclassifier.predict(ct_test).round()

In [76]:
pd.DataFrame(y_test_pred).to_csv("laveena_valecha_project2.csv",header="V86",index=False)